In [1]:
import pandas as pd
import re
import json
pd.options.mode.chained_assignment = None

# Path to files

In [2]:
SCHEDULE_FOLDER = "schedules/"

ECONOMICS_SCHEDULE = "Econ.xlsx"
PE_SCHEDULE = "ProgEng.xlsx"

economics_path =SCHEDULE_FOLDER+ECONOMICS_SCHEDULE
pe_path = SCHEDULE_FOLDER + PE_SCHEDULE

RESULT_JSON = "Result.json"

# Creating tables

## Economics

In [3]:
econ_df = pd.read_excel(economics_path)
econ_df

,День,Час,"Дисципліна, викладач",Unnamed: 3,Група,Unnamed: 5,Unnamed: 6,Тиждень,Ауд.,Unnamed: 9,Unnamed: 10
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,NaN,1,NaN,NaN,1-14,Д,NaN,NaN
1,NaN,NaN,Управління брендом роботодавця (мен.)\n асист....,NaN,лекція,NaN,NaN,3-9,Д,NaN,NaN
2,NaN,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) \n ст....,NaN,лекція,NaN,NaN,2-9,Д,NaN,NaN
3,NaN,NaN,Моделювання та управління фінансовими активами...,NaN,лекція,NaN,NaN,1-11,Д,NaN,NaN
4,NaN,NaN,Маркетинг впливу (мар.) (Custumer experience) ...,NaN,лекція,NaN,NaN,"2,3,4",Д,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
104,NaN,11.40-13.00,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,NaN,2,"2,4,5,6,7,8",NaN,NaN,NaN,Д
105,NaN,NaN,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,NaN,2пр,9-13,NaN,NaN,NaN,Д
106,NaN,13.30-14.50,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,NaN,лекція,2-9,NaN,NaN,NaN,Д
107,NaN,15.00-16.20,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,NaN,3ф,3-9,NaN,NaN,NaN,Д


## Program engineering

In [4]:
pe_df = pd.read_excel(pe_path)
pe_df

,Unnamed: 0,Unnamed: 1,ЗАТВЕРДЖУЮ,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,Декан факультету________А.М. Глибовець,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,""" 24 "" серпня 2023р.",NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,Факультет інформатики,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
67,NaN,13:30-14:50,"Інформаційна безпека веб-застосунків, ас. О.В,...",5,1-10,Дистанційно
68,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,15:00-16:20,"Інформаційна безпека веб-застосунків, ас. О.В,...",6,1-10,Дистанційно
70,NaN,16:30-17:50,NaN,NaN,NaN,NaN


# Preparing dataframes to merge

## Program engineering

### Delete text before the main table

In [5]:
pe_df2 = pe_df.drop(range(0,8))
pe_df2.columns = pe_df2.iloc[0]
pe_df2 = pe_df2.rename_axis(None, axis=1)
pe_df2=pe_df2.drop([8]).reset_index()
pe_df2= pe_df2.drop(columns=['index'])
pe_df2

,День,Час,"Дисципліна, викладач",Група,Тижні,Аудиторія
0,Понеділок,8:30-9:50,NaN,NaN,NaN,NaN
1,NaN,10:00-11:20,"Методи об'єктно-орієнтованого програмування, д...",лекція,1-11,Дистанційно
2,NaN,11:40-13:00,"Методи об'єктно-орієнтованого програмування, д...",1,1-10,Дистанційно
3,NaN,13:30-14:50,"Системне програмування, доц. О.П. Жежерун",лекція,1-10,Дистанційно
4,NaN,15:00-16:20,"Системне програмування, доц. О.П. Жежерун",1,1-11,Дистанційно
...,...,...,...,...,...,...
58,NaN,13:30-14:50,"Інформаційна безпека веб-застосунків, ас. О.В,...",5,1-10,Дистанційно
59,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,15:00-16:20,"Інформаційна безпека веб-застосунків, ас. О.В,...",6,1-10,Дистанційно
61,NaN,16:30-17:50,NaN,NaN,NaN,NaN


### Filling NA values in "День" and "Час"

In [6]:
pe_df2['День']=pe_df2['День'].ffill()
pe_df2['Час']=pe_df2['Час'].ffill()
pe_df2

,День,Час,"Дисципліна, викладач",Група,Тижні,Аудиторія
0,Понеділок,8:30-9:50,NaN,NaN,NaN,NaN
1,Понеділок,10:00-11:20,"Методи об'єктно-орієнтованого програмування, д...",лекція,1-11,Дистанційно
2,Понеділок,11:40-13:00,"Методи об'єктно-орієнтованого програмування, д...",1,1-10,Дистанційно
3,Понеділок,13:30-14:50,"Системне програмування, доц. О.П. Жежерун",лекція,1-10,Дистанційно
4,Понеділок,15:00-16:20,"Системне програмування, доц. О.П. Жежерун",1,1-11,Дистанційно
...,...,...,...,...,...,...
58,Субота,13:30-14:50,"Інформаційна безпека веб-застосунків, ас. О.В,...",5,1-10,Дистанційно
59,Субота,13:30-14:50,NaN,NaN,NaN,NaN
60,Субота,15:00-16:20,"Інформаційна безпека веб-застосунків, ас. О.В,...",6,1-10,Дистанційно
61,Субота,16:30-17:50,NaN,NaN,NaN,NaN


### Remove rows with no lessons

In [7]:
pe_df2=pe_df2.dropna()
pe_df2

,День,Час,"Дисципліна, викладач",Група,Тижні,Аудиторія
1,Понеділок,10:00-11:20,"Методи об'єктно-орієнтованого програмування, д...",лекція,1-11,Дистанційно
2,Понеділок,11:40-13:00,"Методи об'єктно-орієнтованого програмування, д...",1,1-10,Дистанційно
3,Понеділок,13:30-14:50,"Системне програмування, доц. О.П. Жежерун",лекція,1-10,Дистанційно
4,Понеділок,15:00-16:20,"Системне програмування, доц. О.П. Жежерун",1,1-11,Дистанційно
5,Понеділок,16:30-17:50,"Системне програмування, доц. О.П. Жежерун",2,1-11,Дистанційно
6,Понеділок,18:00-19:20,"Системне програмування, доц. О.П. Жежерун",3,1-11,Дистанційно
7,Вівторок,8:30-9:50,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",лекція,1-5,Дистанційно
8,Вівторок,10:00-11:20,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",1,1-8,Дистанційно
9,Вівторок,10:00-11:20,"Комп'ютерна вірусологія, ст.викл. О. М. Пєчкурова",2,1-8,Дистанційно
10,Вівторок,11:40-13:01,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",3,1-8,Дистанційно


### Adding faculty

In [8]:
pe_df2['Факультет'] = "ФІ"
pe_df2

,День,Час,"Дисципліна, викладач",Група,Тижні,Аудиторія,Факультет
1,Понеділок,10:00-11:20,"Методи об'єктно-орієнтованого програмування, д...",лекція,1-11,Дистанційно,ФІ
2,Понеділок,11:40-13:00,"Методи об'єктно-орієнтованого програмування, д...",1,1-10,Дистанційно,ФІ
3,Понеділок,13:30-14:50,"Системне програмування, доц. О.П. Жежерун",лекція,1-10,Дистанційно,ФІ
4,Понеділок,15:00-16:20,"Системне програмування, доц. О.П. Жежерун",1,1-11,Дистанційно,ФІ
5,Понеділок,16:30-17:50,"Системне програмування, доц. О.П. Жежерун",2,1-11,Дистанційно,ФІ
6,Понеділок,18:00-19:20,"Системне програмування, доц. О.П. Жежерун",3,1-11,Дистанційно,ФІ
7,Вівторок,8:30-9:50,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",лекція,1-5,Дистанційно,ФІ
8,Вівторок,10:00-11:20,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",1,1-8,Дистанційно,ФІ
9,Вівторок,10:00-11:20,"Комп'ютерна вірусологія, ст.викл. О. М. Пєчкурова",2,1-8,Дистанційно,ФІ
10,Вівторок,11:40-13:01,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",3,1-8,Дистанційно,ФІ


### Adding speciality

In [9]:
pe_df2['Спеціальність'] = "ІПЗ"
pe_df2

,День,Час,"Дисципліна, викладач",Група,Тижні,Аудиторія,Факультет,Спеціальність
1,Понеділок,10:00-11:20,"Методи об'єктно-орієнтованого програмування, д...",лекція,1-11,Дистанційно,ФІ,ІПЗ
2,Понеділок,11:40-13:00,"Методи об'єктно-орієнтованого програмування, д...",1,1-10,Дистанційно,ФІ,ІПЗ
3,Понеділок,13:30-14:50,"Системне програмування, доц. О.П. Жежерун",лекція,1-10,Дистанційно,ФІ,ІПЗ
4,Понеділок,15:00-16:20,"Системне програмування, доц. О.П. Жежерун",1,1-11,Дистанційно,ФІ,ІПЗ
5,Понеділок,16:30-17:50,"Системне програмування, доц. О.П. Жежерун",2,1-11,Дистанційно,ФІ,ІПЗ
6,Понеділок,18:00-19:20,"Системне програмування, доц. О.П. Жежерун",3,1-11,Дистанційно,ФІ,ІПЗ
7,Вівторок,8:30-9:50,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",лекція,1-5,Дистанційно,ФІ,ІПЗ
8,Вівторок,10:00-11:20,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",1,1-8,Дистанційно,ФІ,ІПЗ
9,Вівторок,10:00-11:20,"Комп'ютерна вірусологія, ст.викл. О. М. Пєчкурова",2,1-8,Дистанційно,ФІ,ІПЗ
10,Вівторок,11:40-13:01,"Комп'ютерна вірусологія, ст.викл. О.В. Кирієнко",3,1-8,Дистанційно,ФІ,ІПЗ


### Split "Дисципліна, викладач" into "Дисципліна" and "Викладач"

In [10]:
pe_df2[['Дисципліна','Викладач']] = pe_df2['Дисципліна, викладач'].str.split(', ',expand=True)
pe_df2=pe_df2.drop(columns='Дисципліна, викладач')
pe_df2 = pe_df2[[
 'Факультет',
 'Спеціальність',
 'Дисципліна',
 'Група',
 'День',
 'Час',
 'Тижні',
 'Аудиторія',
 'Викладач'
]]
pe_df2

,Факультет,Спеціальність,Дисципліна,Група,День,Час,Тижні,Аудиторія,Викладач
1,ФІ,ІПЗ,Методи об'єктно-орієнтованого програмування,лекція,Понеділок,10:00-11:20,1-11,Дистанційно,доц. В.В. Бублик
2,ФІ,ІПЗ,Методи об'єктно-орієнтованого програмування,1,Понеділок,11:40-13:00,1-10,Дистанційно,доц. В.В. Бублик
3,ФІ,ІПЗ,Системне програмування,лекція,Понеділок,13:30-14:50,1-10,Дистанційно,доц. О.П. Жежерун
4,ФІ,ІПЗ,Системне програмування,1,Понеділок,15:00-16:20,1-11,Дистанційно,доц. О.П. Жежерун
5,ФІ,ІПЗ,Системне програмування,2,Понеділок,16:30-17:50,1-11,Дистанційно,доц. О.П. Жежерун
6,ФІ,ІПЗ,Системне програмування,3,Понеділок,18:00-19:20,1-11,Дистанційно,доц. О.П. Жежерун
7,ФІ,ІПЗ,Комп'ютерна вірусологія,лекція,Вівторок,8:30-9:50,1-5,Дистанційно,ст.викл. О.В. Кирієнко
8,ФІ,ІПЗ,Комп'ютерна вірусологія,1,Вівторок,10:00-11:20,1-8,Дистанційно,ст.викл. О.В. Кирієнко
9,ФІ,ІПЗ,Комп'ютерна вірусологія,2,Вівторок,10:00-11:20,1-8,Дистанційно,ст.викл. О. М. Пєчкурова
10,ФІ,ІПЗ,Комп'ютерна вірусологія,3,Вівторок,11:40-13:01,1-8,Дистанційно,ст.викл. О.В. Кирієнко


## Faculty of Economics

### Merging classes

In [11]:
econ_df['Unnamed: 9'] = econ_df['Unnamed: 9'].fillna(econ_df['Unnamed: 10'])
econ_df['Ауд.'] = econ_df['Ауд.'].fillna(econ_df['Unnamed: 9'])
econ_df=econ_df.drop(columns=['Unnamed: 9','Unnamed: 10'])
econ_df

,День,Час,"Дисципліна, викладач",Unnamed: 3,Група,Unnamed: 5,Unnamed: 6,Тиждень,Ауд.
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,NaN,1,NaN,NaN,1-14,Д
1,NaN,NaN,Управління брендом роботодавця (мен.)\n асист....,NaN,лекція,NaN,NaN,3-9,Д
2,NaN,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) \n ст....,NaN,лекція,NaN,NaN,2-9,Д
3,NaN,NaN,Моделювання та управління фінансовими активами...,NaN,лекція,NaN,NaN,1-11,Д
4,NaN,NaN,Маркетинг впливу (мар.) (Custumer experience) ...,NaN,лекція,NaN,NaN,"2,3,4",Д
...,...,...,...,...,...,...,...,...,...
104,NaN,11.40-13.00,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,NaN,2,"2,4,5,6,7,8",NaN,Д
105,NaN,NaN,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,NaN,2пр,9-13,NaN,Д
106,NaN,13.30-14.50,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,NaN,лекція,2-9,NaN,Д
107,NaN,15.00-16.20,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,NaN,3ф,3-9,NaN,Д


### Merging weeks

In [12]:
econ_df['Тиждень'] = econ_df['Тиждень'].fillna(econ_df['Unnamed: 6'])
econ_df=econ_df.drop(columns=['Unnamed: 6'])
econ_df

,День,Час,"Дисципліна, викладач",Unnamed: 3,Група,Unnamed: 5,Тиждень,Ауд.
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,NaN,1,NaN,1-14,Д
1,NaN,NaN,Управління брендом роботодавця (мен.)\n асист....,NaN,лекція,NaN,3-9,Д
2,NaN,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) \n ст....,NaN,лекція,NaN,2-9,Д
3,NaN,NaN,Моделювання та управління фінансовими активами...,NaN,лекція,NaN,1-11,Д
4,NaN,NaN,Маркетинг впливу (мар.) (Custumer experience) ...,NaN,лекція,NaN,"2,3,4",Д
...,...,...,...,...,...,...,...,...
104,NaN,11.40-13.00,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,NaN,2,"2,4,5,6,7,8",Д
105,NaN,NaN,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,NaN,2пр,9-13,Д
106,NaN,13.30-14.50,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,NaN,лекція,2-9,Д
107,NaN,15.00-16.20,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,NaN,3ф,3-9,Д


### Merging groups

In [13]:
econ_df['Група'] = econ_df['Група'].fillna(econ_df['Unnamed: 5'])
econ_df=econ_df.drop(columns=['Unnamed: 5'])
econ_df

,День,Час,"Дисципліна, викладач",Unnamed: 3,Група,Тиждень,Ауд.
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,NaN,1,1-14,Д
1,NaN,NaN,Управління брендом роботодавця (мен.)\n асист....,NaN,лекція,3-9,Д
2,NaN,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) \n ст....,NaN,лекція,2-9,Д
3,NaN,NaN,Моделювання та управління фінансовими активами...,NaN,лекція,1-11,Д
4,NaN,NaN,Маркетинг впливу (мар.) (Custumer experience) ...,NaN,лекція,"2,3,4",Д
...,...,...,...,...,...,...,...
104,NaN,11.40-13.00,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,2,"2,4,5,6,7,8",Д
105,NaN,NaN,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,2пр,9-13,Д
106,NaN,13.30-14.50,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,лекція,2-9,Д
107,NaN,15.00-16.20,NaN,Економіка підприємства (фін.) \n ст. викл. Хра...,3ф,3-9,Д


### Merging courses

In [14]:
econ_df['Дисципліна, викладач'] = econ_df['Дисципліна, викладач'].fillna(econ_df['Unnamed: 3'])
econ_df=econ_df.drop(columns=['Unnamed: 3'])
econ_df

,День,Час,"Дисципліна, викладач",Група,Тиждень,Ауд.
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,1,1-14,Д
1,NaN,NaN,Управління брендом роботодавця (мен.)\n асист....,лекція,3-9,Д
2,NaN,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) \n ст....,лекція,2-9,Д
3,NaN,NaN,Моделювання та управління фінансовими активами...,лекція,1-11,Д
4,NaN,NaN,Маркетинг впливу (мар.) (Custumer experience) ...,лекція,"2,3,4",Д
...,...,...,...,...,...,...
104,NaN,11.40-13.00,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,2,"2,4,5,6,7,8",Д
105,NaN,NaN,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,2пр,9-13,Д
106,NaN,13.30-14.50,Економіка підприємства (фін.) \n ст. викл. Хра...,лекція,2-9,Д
107,NaN,15.00-16.20,Економіка підприємства (фін.) \n ст. викл. Хра...,3ф,3-9,Д


### Filling NA values in “День” and “Час”

In [15]:
econ_df['День']=econ_df['День'].ffill()
econ_df['Час']=econ_df['Час'].ffill()
econ_df

,День,Час,"Дисципліна, викладач",Група,Тиждень,Ауд.
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,1,1-14,Д
1,Понеділок,8.30-9.50,Управління брендом роботодавця (мен.)\n асист....,лекція,3-9,Д
2,Понеділок,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) \n ст....,лекція,2-9,Д
3,Понеділок,10.00-11.20,Моделювання та управління фінансовими активами...,лекція,1-11,Д
4,Понеділок,10.00-11.20,Маркетинг впливу (мар.) (Custumer experience) ...,лекція,"2,3,4",Д
...,...,...,...,...,...,...
104,Субота,11.40-13.00,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,2,"2,4,5,6,7,8",Д
105,Субота,11.40-13.00,Комп’ютерна бізнес-статистика \n доц. Братик М.В.,2пр,9-13,Д
106,Субота,13.30-14.50,Економіка підприємства (фін.) \n ст. викл. Хра...,лекція,2-9,Д
107,Субота,15.00-16.20,Економіка підприємства (фін.) \n ст. викл. Хра...,3ф,3-9,Д


### Removing '\n' from courses and groups

In [16]:
econ_df['Дисципліна, викладач'] = econ_df['Дисципліна, викладач'].str.replace('\n', '')
econ_df['Група'] = econ_df['Група'].str.replace('\n', '')
econ_df

,День,Час,"Дисципліна, викладач",Група,Тиждень,Ауд.
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,1,1-14,Д
1,Понеділок,8.30-9.50,Управління брендом роботодавця (мен.) асист. Ю...,лекція,3-9,Д
2,Понеділок,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) ст.ви...,лекція,2-9,Д
3,Понеділок,10.00-11.20,Моделювання та управління фінансовими активами...,лекція,1-11,Д
4,Понеділок,10.00-11.20,Маркетинг впливу (мар.) (Custumer experience) ...,лекція,"2,3,4",Д
...,...,...,...,...,...,...
104,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2,"2,4,5,6,7,8",Д
105,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2пр,9-13,Д
106,Субота,13.30-14.50,Економіка підприємства (фін.) ст. викл. Храба...,лекція,2-9,Д
107,Субота,15.00-16.20,Економіка підприємства (фін.) ст. викл. Храба...,3ф,3-9,Д


### Remove rows with no lessons

In [17]:
econ_df=econ_df.dropna()
econ_df

,День,Час,"Дисципліна, викладач",Група,Тиждень,Ауд.
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,1,1-14,Д
1,Понеділок,8.30-9.50,Управління брендом роботодавця (мен.) асист. Ю...,лекція,3-9,Д
2,Понеділок,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) ст.ви...,лекція,2-9,Д
3,Понеділок,10.00-11.20,Моделювання та управління фінансовими активами...,лекція,1-11,Д
4,Понеділок,10.00-11.20,Маркетинг впливу (мар.) (Custumer experience) ...,лекція,"2,3,4",Д
...,...,...,...,...,...,...
104,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2,"2,4,5,6,7,8",Д
105,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2пр,9-13,Д
106,Субота,13.30-14.50,Економіка підприємства (фін.) ст. викл. Храба...,лекція,2-9,Д
107,Субота,15.00-16.20,Економіка підприємства (фін.) ст. викл. Храба...,3ф,3-9,Д


### Split “Дисципліна, викладач” into “Дисципліна” and “Викладач”

In [18]:
econ_df["Дисципліна"] = econ_df['Дисципліна, викладач'].str.split('асист.|проф.|доц.|ст.викл|ст. викл', expand=True)[0]

In [19]:
econ_df

,День,Час,"Дисципліна, викладач",Група,Тиждень,Ауд.,Дисципліна
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,1,1-14,Д,Навчально-науковий семінар з економіки (екон.)
1,Понеділок,8.30-9.50,Управління брендом роботодавця (мен.) асист. Ю...,лекція,3-9,Д,Управління брендом роботодавця (мен.)
2,Понеділок,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) ст.ви...,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.)
3,Понеділок,10.00-11.20,Моделювання та управління фінансовими активами...,лекція,1-11,Д,Моделювання та управління фінансовими активами...
4,Понеділок,10.00-11.20,Маркетинг впливу (мар.) (Custumer experience) ...,лекція,"2,3,4",Д,Маркетинг впливу (мар.) (Custumer experience)
...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика
105,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2пр,9-13,Д,Комп’ютерна бізнес-статистика
106,Субота,13.30-14.50,Економіка підприємства (фін.) ст. викл. Храба...,лекція,2-9,Д,Економіка підприємства (фін.)
107,Субота,15.00-16.20,Економіка підприємства (фін.) ст. викл. Храба...,3ф,3-9,Д,Економіка підприємства (фін.)


In [20]:
econ_df['Викладач'] = [b.replace(a, '') for a,b in zip(econ_df["Дисципліна"], econ_df['Дисципліна, викладач'])]
econ_df

,День,Час,"Дисципліна, викладач",Група,Тиждень,Ауд.,Дисципліна,Викладач
0,Понеділок,8.30-9.50,Навчально-науковий семінар з економіки (екон.)...,1,1-14,Д,Навчально-науковий семінар з економіки (екон.),проф. Бураковський І.В.
1,Понеділок,8.30-9.50,Управління брендом роботодавця (мен.) асист. Ю...,лекція,3-9,Д,Управління брендом роботодавця (мен.),асист. Юрченко Я.В.
2,Понеділок,10.00-11.20,Макро- та мікроеконом. моделюв. (екон.) ст.ви...,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.),ст.викл. Яценко Г.Ю.
3,Понеділок,10.00-11.20,Моделювання та управління фінансовими активами...,лекція,1-11,Д,Моделювання та управління фінансовими активами...,проф. Долінський Л.Б.
4,Понеділок,10.00-11.20,Маркетинг впливу (мар.) (Custumer experience) ...,лекція,"2,3,4",Д,Маркетинг впливу (мар.) (Custumer experience),ст.викл. Мехед Т.М..
...,...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.
105,Субота,11.40-13.00,Комп’ютерна бізнес-статистика доц. Братик М.В.,2пр,9-13,Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.
106,Субота,13.30-14.50,Економіка підприємства (фін.) ст. викл. Храба...,лекція,2-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.
107,Субота,15.00-16.20,Економіка підприємства (фін.) ст. викл. Храба...,3ф,3-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.


In [21]:
econ_df=econ_df.drop(columns=['Дисципліна, викладач'])
econ_df

,День,Час,Група,Тиждень,Ауд.,Дисципліна,Викладач
0,Понеділок,8.30-9.50,1,1-14,Д,Навчально-науковий семінар з економіки (екон.),проф. Бураковський І.В.
1,Понеділок,8.30-9.50,лекція,3-9,Д,Управління брендом роботодавця (мен.),асист. Юрченко Я.В.
2,Понеділок,10.00-11.20,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.),ст.викл. Яценко Г.Ю.
3,Понеділок,10.00-11.20,лекція,1-11,Д,Моделювання та управління фінансовими активами...,проф. Долінський Л.Б.
4,Понеділок,10.00-11.20,лекція,"2,3,4",Д,Маркетинг впливу (мар.) (Custumer experience),ст.викл. Мехед Т.М..
...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.
105,Субота,11.40-13.00,2пр,9-13,Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.
106,Субота,13.30-14.50,лекція,2-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.
107,Субота,15.00-16.20,3ф,3-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.


### Separating specialty

In [22]:
# 'Маркетинг впливу (Custumer experience) (мар.)' is unique case, because specialty in brackets before another brackets
econ_df.loc[econ_df['Дисципліна'].str.contains('Маркетинг впливу'), 'Дисципліна']

4     Маркетинг впливу (мар.) (Custumer experience) 
5     Маркетинг впливу (мар.) (Custumer experience) 
9     Маркетинг впливу (мар.) (Custumer experience) 
10    Маркетинг впливу (мар.) (Custumer experience) 
13    Маркетинг впливу (мар.) (Custumer experience) 
Name: Дисципліна, dtype: object

In [23]:
econ_df.loc[econ_df['Дисципліна'].str.contains('Маркетинг впливу'), 'Дисципліна'] = 'Маркетинг впливу (Custumer experience) (мар.)'
econ_df

,День,Час,Група,Тиждень,Ауд.,Дисципліна,Викладач
0,Понеділок,8.30-9.50,1,1-14,Д,Навчально-науковий семінар з економіки (екон.),проф. Бураковський І.В.
1,Понеділок,8.30-9.50,лекція,3-9,Д,Управління брендом роботодавця (мен.),асист. Юрченко Я.В.
2,Понеділок,10.00-11.20,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.),ст.викл. Яценко Г.Ю.
3,Понеділок,10.00-11.20,лекція,1-11,Д,Моделювання та управління фінансовими активами...,проф. Долінський Л.Б.
4,Понеділок,10.00-11.20,лекція,"2,3,4",Д,Маркетинг впливу (Custumer experience) (мар.),ст.викл. Мехед Т.М..
...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.
105,Субота,11.40-13.00,2пр,9-13,Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.
106,Субота,13.30-14.50,лекція,2-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.
107,Субота,15.00-16.20,3ф,3-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.


In [24]:
#taking specialty from last brackets, special cases will be fixed further in code
econ_df['Спеціальність'] = econ_df['Дисципліна'].apply(lambda x: re.sub('^.*\((.*?)\)[^\(]*$', '\g<1>', x))
econ_df

,День,Час,Група,Тиждень,Ауд.,Дисципліна,Викладач,Спеціальність
0,Понеділок,8.30-9.50,1,1-14,Д,Навчально-науковий семінар з економіки (екон.),проф. Бураковський І.В.,екон.
1,Понеділок,8.30-9.50,лекція,3-9,Д,Управління брендом роботодавця (мен.),асист. Юрченко Я.В.,мен.
2,Понеділок,10.00-11.20,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.),ст.викл. Яценко Г.Ю.,екон.
3,Понеділок,10.00-11.20,лекція,1-11,Д,Моделювання та управління фінансовими активами...,проф. Долінський Л.Б.,фін.
4,Понеділок,10.00-11.20,лекція,"2,3,4",Д,Маркетинг впливу (Custumer experience) (мар.),ст.викл. Мехед Т.М..,мар.
...,...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,Комп’ютерна бізнес-статистика
105,Субота,11.40-13.00,2пр,9-13,Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,Комп’ютерна бізнес-статистика
106,Субота,13.30-14.50,лекція,2-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін.
107,Субота,15.00-16.20,3ф,3-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін.


In [25]:
#special case because it's available for all specialties
econ_df.loc[econ_df['Дисципліна'].str.contains('економ. теор.'), 'Спеціальність'] = 'фін+екон+мен+мар'
econ_df

,День,Час,Група,Тиждень,Ауд.,Дисципліна,Викладач,Спеціальність
0,Понеділок,8.30-9.50,1,1-14,Д,Навчально-науковий семінар з економіки (екон.),проф. Бураковський І.В.,екон.
1,Понеділок,8.30-9.50,лекція,3-9,Д,Управління брендом роботодавця (мен.),асист. Юрченко Я.В.,мен.
2,Понеділок,10.00-11.20,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.),ст.викл. Яценко Г.Ю.,екон.
3,Понеділок,10.00-11.20,лекція,1-11,Д,Моделювання та управління фінансовими активами...,проф. Долінський Л.Б.,фін.
4,Понеділок,10.00-11.20,лекція,"2,3,4",Д,Маркетинг впливу (Custumer experience) (мар.),ст.викл. Мехед Т.М..,мар.
...,...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,Комп’ютерна бізнес-статистика
105,Субота,11.40-13.00,2пр,9-13,Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,Комп’ютерна бізнес-статистика
106,Субота,13.30-14.50,лекція,2-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін.
107,Субота,15.00-16.20,3ф,3-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін.


In [26]:
#special case because it's available for all specialties
econ_df.loc[econ_df['Дисципліна'].str.contains('Комп’ютерна бізнес-статистика'), 'Спеціальність'] = 'фін+екон+мен+мар'
econ_df

,День,Час,Група,Тиждень,Ауд.,Дисципліна,Викладач,Спеціальність
0,Понеділок,8.30-9.50,1,1-14,Д,Навчально-науковий семінар з економіки (екон.),проф. Бураковський І.В.,екон.
1,Понеділок,8.30-9.50,лекція,3-9,Д,Управління брендом роботодавця (мен.),асист. Юрченко Я.В.,мен.
2,Понеділок,10.00-11.20,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.),ст.викл. Яценко Г.Ю.,екон.
3,Понеділок,10.00-11.20,лекція,1-11,Д,Моделювання та управління фінансовими активами...,проф. Долінський Л.Б.,фін.
4,Понеділок,10.00-11.20,лекція,"2,3,4",Д,Маркетинг впливу (Custumer experience) (мар.),ст.викл. Мехед Т.М..,мар.
...,...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,фін+екон+мен+мар
105,Субота,11.40-13.00,2пр,9-13,Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,фін+екон+мен+мар
106,Субота,13.30-14.50,лекція,2-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін.
107,Субота,15.00-16.20,3ф,3-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін.


In [27]:
#preparing to explode rows with multiple specialties
econ_df['Спеціальність'] = econ_df['Спеціальність'].replace('\.','', regex=True).replace(', ',',', regex=True).replace(',','+', regex=True)
econ_df

,День,Час,Група,Тиждень,Ауд.,Дисципліна,Викладач,Спеціальність
0,Понеділок,8.30-9.50,1,1-14,Д,Навчально-науковий семінар з економіки (екон.),проф. Бураковський І.В.,екон
1,Понеділок,8.30-9.50,лекція,3-9,Д,Управління брендом роботодавця (мен.),асист. Юрченко Я.В.,мен
2,Понеділок,10.00-11.20,лекція,2-9,Д,Макро- та мікроеконом. моделюв. (екон.),ст.викл. Яценко Г.Ю.,екон
3,Понеділок,10.00-11.20,лекція,1-11,Д,Моделювання та управління фінансовими активами...,проф. Долінський Л.Б.,фін
4,Понеділок,10.00-11.20,лекція,"2,3,4",Д,Маркетинг впливу (Custumer experience) (мар.),ст.викл. Мехед Т.М..,мар
...,...,...,...,...,...,...,...,...
104,Субота,11.40-13.00,2,"2,4,5,6,7,8",Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,фін+екон+мен+мар
105,Субота,11.40-13.00,2пр,9-13,Д,Комп’ютерна бізнес-статистика,доц. Братик М.В.,фін+екон+мен+мар
106,Субота,13.30-14.50,лекція,2-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін
107,Субота,15.00-16.20,3ф,3-9,Д,Економіка підприємства (фін.),ст. викл. Храбан А.М.,фін


In [28]:
#preparing to explode rows with multiple specialties
econ_df['Спеціальність'] = econ_df['Спеціальність'].str.split('+')
econ_df=econ_df.explode(column='Спеціальність')

In [29]:
#explode rows with multiple specialties
econ_df['Факультет'] = "ФЕН"
econ_df=econ_df.rename(columns={'Тиждень':'Тижні', 'Ауд.':'Аудиторія'})
econ_df = econ_df[[
 'Факультет',
 'Спеціальність',
 'Дисципліна',
 'Група',
 'День',
 'Час',
 'Тижні',
 'Аудиторія',
 'Викладач'
]]
econ_df

,Факультет,Спеціальність,Дисципліна,Група,День,Час,Тижні,Аудиторія,Викладач
0,ФЕН,екон,Навчально-науковий семінар з економіки (екон.),1,Понеділок,8.30-9.50,1-14,Д,проф. Бураковський І.В.
1,ФЕН,мен,Управління брендом роботодавця (мен.),лекція,Понеділок,8.30-9.50,3-9,Д,асист. Юрченко Я.В.
2,ФЕН,екон,Макро- та мікроеконом. моделюв. (екон.),лекція,Понеділок,10.00-11.20,2-9,Д,ст.викл. Яценко Г.Ю.
3,ФЕН,фін,Моделювання та управління фінансовими активами...,лекція,Понеділок,10.00-11.20,1-11,Д,проф. Долінський Л.Б.
4,ФЕН,мар,Маркетинг впливу (Custumer experience) (мар.),лекція,Понеділок,10.00-11.20,"2,3,4",Д,ст.викл. Мехед Т.М..
...,...,...,...,...,...,...,...,...,...
105,ФЕН,мен,Комп’ютерна бізнес-статистика,2пр,Субота,11.40-13.00,9-13,Д,доц. Братик М.В.
105,ФЕН,мар,Комп’ютерна бізнес-статистика,2пр,Субота,11.40-13.00,9-13,Д,доц. Братик М.В.
106,ФЕН,фін,Економіка підприємства (фін.),лекція,Субота,13.30-14.50,2-9,Д,ст. викл. Храбан А.М.
107,ФЕН,фін,Економіка підприємства (фін.),3ф,Субота,15.00-16.20,3-9,Д,ст. викл. Храбан А.М.


In [30]:
#replacing short names of specialties with full versions
econ_df['Спеціальність'] = econ_df['Спеціальність'].apply(lambda x: x[:2])
def full_specialty(short_specialty):
 if(short_specialty=='ма'): return "Маркетинг"
 elif(short_specialty=='ек'): return "Економіка"
 elif(short_specialty=='фі'): return "Фінанси"
 else: return "Менеджмент"
econ_df['Спеціальність'] = econ_df['Спеціальність'].apply(lambda x: full_specialty(x))
econ_df

,Факультет,Спеціальність,Дисципліна,Група,День,Час,Тижні,Аудиторія,Викладач
0,ФЕН,Економіка,Навчально-науковий семінар з економіки (екон.),1,Понеділок,8.30-9.50,1-14,Д,проф. Бураковський І.В.
1,ФЕН,Менеджмент,Управління брендом роботодавця (мен.),лекція,Понеділок,8.30-9.50,3-9,Д,асист. Юрченко Я.В.
2,ФЕН,Економіка,Макро- та мікроеконом. моделюв. (екон.),лекція,Понеділок,10.00-11.20,2-9,Д,ст.викл. Яценко Г.Ю.
3,ФЕН,Фінанси,Моделювання та управління фінансовими активами...,лекція,Понеділок,10.00-11.20,1-11,Д,проф. Долінський Л.Б.
4,ФЕН,Маркетинг,Маркетинг впливу (Custumer experience) (мар.),лекція,Понеділок,10.00-11.20,"2,3,4",Д,ст.викл. Мехед Т.М..
...,...,...,...,...,...,...,...,...,...
105,ФЕН,Менеджмент,Комп’ютерна бізнес-статистика,2пр,Субота,11.40-13.00,9-13,Д,доц. Братик М.В.
105,ФЕН,Маркетинг,Комп’ютерна бізнес-статистика,2пр,Субота,11.40-13.00,9-13,Д,доц. Братик М.В.
106,ФЕН,Фінанси,Економіка підприємства (фін.),лекція,Субота,13.30-14.50,2-9,Д,ст. викл. Храбан А.М.
107,ФЕН,Фінанси,Економіка підприємства (фін.),3ф,Субота,15.00-16.20,3-9,Д,ст. викл. Храбан А.М.


# Merging dataset

In [31]:
merged = pd.concat([econ_df, pe_df2], axis=0).reset_index()
merged=merged.drop(columns='index')
merged

,Факультет,Спеціальність,Дисципліна,Група,День,Час,Тижні,Аудиторія,Викладач
0,ФЕН,Економіка,Навчально-науковий семінар з економіки (екон.),1,Понеділок,8.30-9.50,1-14,Д,проф. Бураковський І.В.
1,ФЕН,Менеджмент,Управління брендом роботодавця (мен.),лекція,Понеділок,8.30-9.50,3-9,Д,асист. Юрченко Я.В.
2,ФЕН,Економіка,Макро- та мікроеконом. моделюв. (екон.),лекція,Понеділок,10.00-11.20,2-9,Д,ст.викл. Яценко Г.Ю.
3,ФЕН,Фінанси,Моделювання та управління фінансовими активами...,лекція,Понеділок,10.00-11.20,1-11,Д,проф. Долінський Л.Б.
4,ФЕН,Маркетинг,Маркетинг впливу (Custumer experience) (мар.),лекція,Понеділок,10.00-11.20,"2,3,4",Д,ст.викл. Мехед Т.М..
...,...,...,...,...,...,...,...,...,...
225,ФІ,ІПЗ,Схематотехніка,1,Субота,10:00-11:20,2-10,Дистанційно,cт. викл..Я.І. Вознюк
226,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,4,Субота,11:40-13:00,1-10,Дистанційно,"ас. О.В,Радзієвська"
227,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,5,Субота,13:30-14:50,1-10,Дистанційно,"ас. О.В,Радзієвська"
228,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,6,Субота,15:00-16:20,1-10,Дистанційно,"ас. О.В,Радзієвська"


### Merging 'День', 'Час', 'Тижні', 'Аудиторія', 'Викладач' into one column

In [32]:
columns = ['День', 'Час', 'Тижні', 'Аудиторія', 'Викладач']
merged['metadata'] = merged[columns].to_dict(orient='records')
merged = merged.drop(columns=columns)
merged

,Факультет,Спеціальність,Дисципліна,Група,metadata
0,ФЕН,Економіка,Навчально-науковий семінар з економіки (екон.),1,"{'День': 'Понеділок', 'Час': '8.30-9.50', 'Тиж..."
1,ФЕН,Менеджмент,Управління брендом роботодавця (мен.),лекція,"{'День': 'Понеділок', 'Час': '8.30-9.50', 'Тиж..."
2,ФЕН,Економіка,Макро- та мікроеконом. моделюв. (екон.),лекція,"{'День': 'Понеділок', 'Час': '10.00-11.20', 'Т..."
3,ФЕН,Фінанси,Моделювання та управління фінансовими активами...,лекція,"{'День': 'Понеділок', 'Час': '10.00-11.20', 'Т..."
4,ФЕН,Маркетинг,Маркетинг впливу (Custumer experience) (мар.),лекція,"{'День': 'Понеділок', 'Час': '10.00-11.20', 'Т..."
...,...,...,...,...,...
225,ФІ,ІПЗ,Схематотехніка,1,"{'День': 'Субота', 'Час': '10:00-11:20', 'Тижн..."
226,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,4,"{'День': 'Субота', 'Час': '11:40-13:00', 'Тижн..."
227,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,5,"{'День': 'Субота', 'Час': '13:30-14:50', 'Тижн..."
228,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,6,"{'День': 'Субота', 'Час': '15:00-16:20', 'Тижн..."


### Merging 'Група', 'metadata' into one column with dictionaries

In [33]:
def convert(lst):
   res_dict = {}
   for i in range(0, len(lst), 2):
       res_dict[lst[i]] = lst[i + 1]
   return res_dict

columns = ['Група', 'metadata']
merged['Група та дані'] = merged[columns].values.tolist()
merged['Група та дані'] = merged['Група та дані'].apply(lambda x: convert(x))
merged=merged.drop(columns=columns)
merged

,Факультет,Спеціальність,Дисципліна,Група та дані
0,ФЕН,Економіка,Навчально-науковий семінар з економіки (екон.),"{'1': {'День': 'Понеділок', 'Час': '8.30-9.50'..."
1,ФЕН,Менеджмент,Управління брендом роботодавця (мен.),"{'лекція': {'День': 'Понеділок', 'Час': '8.30-..."
2,ФЕН,Економіка,Макро- та мікроеконом. моделюв. (екон.),"{'лекція': {'День': 'Понеділок', 'Час': '10.00..."
3,ФЕН,Фінанси,Моделювання та управління фінансовими активами...,"{'лекція': {'День': 'Понеділок', 'Час': '10.00..."
4,ФЕН,Маркетинг,Маркетинг впливу (Custumer experience) (мар.),"{'лекція': {'День': 'Понеділок', 'Час': '10.00..."
...,...,...,...,...
225,ФІ,ІПЗ,Схематотехніка,"{1: {'День': 'Субота', 'Час': '10:00-11:20', '..."
226,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,"{4: {'День': 'Субота', 'Час': '11:40-13:00', '..."
227,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,"{5: {'День': 'Субота', 'Час': '13:30-14:50', '..."
228,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,"{6: {'День': 'Субота', 'Час': '15:00-16:20', '..."


### Merging groups for subject from specialty and Faculty

In [34]:
merged = merged.groupby(['Факультет','Спеціальність','Дисципліна']).apply(lambda x: x['Група та дані'].values.tolist()).reset_index(name='Групи та дані')
merged

,Факультет,Спеціальність,Дисципліна,Групи та дані
0,ФІ,ІПЗ,Інструменти та принципи веб-розробки,"[{'лекція': {'День': 'Середа', 'Час': '8:30-9:..."
1,ФІ,ІПЗ,Інформаційна безпека веб-застосунків,"[{10: {'День': 'Середа', 'Час': '11:40-13:00',..."
2,ФІ,ІПЗ,Комп'ютерна вірусологія,"[{'лекція': {'День': 'Вівторок', 'Час': '8:30-..."
3,ФІ,ІПЗ,Методи об'єктно-орієнтованого програмування,"[{'лекція': {'День': 'Понеділок', 'Час': '10:0..."
4,ФІ,ІПЗ,Мова програмування Swift,"[{3: {'День': 'Середа', 'Час': '11:40-13:00', ..."
5,ФІ,ІПЗ,Основи роботи з фреймворком Spring Boot,"[{'лекція': {'День': 'Четвер', 'Час': '8:30-9:..."
6,ФІ,ІПЗ,Пошукова оптимізація Веб-застосувань,"[{'лекція': {'День': 'Вівторок', 'Час': '13:30..."
7,ФІ,ІПЗ,Розробка користувацького інтерфейсу (UI/UX),"[{'лекція': {'День': 'Четвер', 'Час': '13:30-1..."
8,ФІ,ІПЗ,Системне програмування,"[{'лекція': {'День': 'Понеділок', 'Час': '13:3..."
9,ФІ,ІПЗ,Схематотехніка,"[{'лекція': {'День': 'Субота', 'Час': '8:30-9:..."


In [35]:
columns = ['Дисципліна', 'Групи та дані']
merged['Дисципліна, групи та дані'] = merged[columns].values.tolist();
merged['Дисципліна, групи та дані'] = merged['Дисципліна, групи та дані'].apply(lambda x: convert(x))
merged=merged.drop(columns = columns)
merged

,Факультет,Спеціальність,"Дисципліна, групи та дані"
0,ФІ,ІПЗ,{'Інструменти та принципи веб-розробки': [{'ле...
1,ФІ,ІПЗ,{'Інформаційна безпека веб-застосунків': [{10:...
2,ФІ,ІПЗ,{'Комп'ютерна вірусологія': [{'лекція': {'День...
3,ФІ,ІПЗ,{'Методи об'єктно-орієнтованого програмування'...
4,ФІ,ІПЗ,{'Мова програмування Swift': [{3: {'День': 'Се...
5,ФІ,ІПЗ,{'Основи роботи з фреймворком Spring Boot': [{...
6,ФІ,ІПЗ,{'Пошукова оптимізація Веб-застосувань': [{'ле...
7,ФІ,ІПЗ,{'Розробка користувацького інтерфейсу (UI/UX)'...
8,ФІ,ІПЗ,{'Системне програмування': [{'лекція': {'День'...
9,ФІ,ІПЗ,{'Схематотехніка': [{'лекція': {'День': 'Субот...


### Merging all subjects with required specialties


In [36]:
merged = merged.groupby(['Факультет','Спеціальність']).apply(lambda x: x['Дисципліна, групи та дані'].values.tolist()).reset_index(name='Дисципліни, групи та дані')
merged

,Факультет,Спеціальність,"Дисципліни, групи та дані"
0,ФІ,ІПЗ,[{'Інструменти та принципи веб-розробки': [{'л...
1,ФЕН,Економіка,[{'Інноваційна модель економ. розвитку (екон.)...
2,ФЕН,Маркетинг,[{'Digital – маркетинг (марк.) ': [{'лекція': ...
3,ФЕН,Менеджмент,"[{'Projekt менеджмент (марк, мен) ': [{'лекці..."
4,ФЕН,Фінанси,[{'Інвестування (фін.) ': [{'2': {'День': 'Сер...


In [37]:
columns = ['Спеціальність', 'Дисципліни, групи та дані']
merged['Спеціальність та дисципліни'] = merged[columns].values.tolist();
merged['Спеціальність та дисципліни'] = merged['Спеціальність та дисципліни'].apply(lambda x: convert(x))
merged=merged.drop(columns = columns)
merged

,Факультет,Спеціальність та дисципліни
0,ФІ,{'ІПЗ': [{'Інструменти та принципи веб-розробк...
1,ФЕН,{'Економіка': [{'Інноваційна модель економ. ро...
2,ФЕН,{'Маркетинг': [{'Digital – маркетинг (марк.) '...
3,ФЕН,"{'Менеджмент': [{'Projekt менеджмент (марк, ме..."
4,ФЕН,{'Фінанси': [{'Інвестування (фін.) ': [{'2': {...


### Merging specialties with required faculty

In [38]:
merged = merged.groupby(['Факультет']).apply(lambda x: x['Спеціальність та дисципліни'].values.tolist()).reset_index(name='Спеціальності та предмети')
merged

,Факультет,Спеціальності та предмети
0,ФІ,[{'ІПЗ': [{'Інструменти та принципи веб-розроб...
1,ФЕН,[{'Економіка': [{'Інноваційна модель економ. р...


# Export resulting json

In [39]:
jsonRes = {}
for i in range(len(merged)):
 jsonRes.update({merged.loc[i, 'Факультет']: merged.loc[i, "Спеціальності та предмети"]})
with open(RESULT_JSON, 'w', encoding='utf8') as json_file:
    json.dump(jsonRes, json_file, ensure_ascii=False, indent=4)